<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/btctxt55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
import bitcoin

# Charger toutes les adresses du fichier btc.txt dans un ensemble
with open('btc.txt', 'r') as file:
    addresses_set = set(line.strip() for line in file)

def is_address_in_set(address):
    """Vérifie si une adresse est présente dans l'ensemble des adresses."""
    return address in addresses_set

def generate_bitcoin_keypair_with_prefix(prefixes):
    attempt_count = 0  # Compteur pour le nombre d'essais
    file_count = 1  # Compteur pour les fichiers
    entries_in_current_file = 0  # Nombre d'entrées dans le fichier actuel

    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)

        # Générer une adresse P2PKH
        address_p2pkh = bitcoin.pubtoaddr(public_key_uncompressed)
        if address_p2pkh.startswith("1") and "1" in prefixes:
            address_type = "P2PKH"
            address = address_p2pkh
        # Générer une adresse P2SH
        elif address_p2pkh.startswith("3") and "3" in prefixes:
            address_type = "P2SH"
            address = address_p2pkh
        # Générer une adresse SegWit P2WPKH
        else:
            address_segwit = bitcoin.pubtosegwitaddr(public_key_uncompressed)
            if address_segwit.startswith("bc1") and "bc1" in prefixes:
                address_type = "SegWit P2WPKH"
                address = address_segwit
            else:
                print(f"Adresse essayée : {address_segwit}")
                continue

        attempt_count += 1
        entries_in_current_file += 1
        print(f"Adresse essayée : {address}")

        # Écrire dans random_N.txt tous les 50 000 essais
        if entries_in_current_file > 50000:
            entries_in_current_file = 1
            file_count += 1

        wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')
        with open(f'random_{file_count}.txt', 'a') as rand_file:
            rand_file.write(f"Essai n°{attempt_count}\n")
            rand_file.write(f"Clé privée hex : {private_key}\n")
            rand_file.write(f"Clé privée WIF : {wif_private_key}\n")
            rand_file.write(f"Adresse ({address_type}) : {address}\n\n")

        if is_address_in_set(address):
            # Si une correspondance est trouvée, on écrit les détails dans resultat.txt
            with open('resultat.txt', 'a') as file:
                file.write(f"Clé privée hex : {private_key}\n")
                file.write(f"Clé privée WIF : {wif_private_key}\n")
                file.write(f"Adresse ({address_type}) : {address}\n\n")
            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "address_type": address_type,
                "address": address
            }

prefixes_to_check = ["1", "3", "bc1"]
kp = generate_bitcoin_keypair_with_prefix(prefixes_to_check)

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print(f"Adresse ({kp['address_type']}) :", kp["address"])
